In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint
import time
import json

In [5]:
#instantiate the browser driver
s=Service(ChromeDriverManager().install())
o=Options()
o.add_argument("start-maximized")
driver = webdriver.Chrome(service=s, options=o)

In [6]:
#Search URL with UAE as a filter
url = "https://www.fragrantica.com/search/?country=United%20Arab%20Emirates"
driver.get(url)
time.sleep(10)

# click load more results to load more perfume details
for i in range(0,1):
    try:
        driver.find_element(By.XPATH,'//button[normalize-space()="Show more results"]').click()
        time.sleep(10)
    except Exception as e:
        print("An exception occurred:", e)
        break

# Get the page html 
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Get the list of perfume names
perfumeGrid = soup.find("span", class_="grid-x grid-margin-x grid-margin-y small-up-3 medium-up-2 large-up-4 perfumes-row text-center")
DictsList = []
Dict = {}
for str in perfumeGrid.find_all("a", href=True):
    perfumeName = str.get_text().replace('\n', '').strip(" ")
    perfumeURL= str['href']
    Dict = {"name": perfumeName,
            "url": perfumeURL
            }
    DictsList.append(Dict)

# Open a file in write mode and add the DictsList to it
with open('DictsList.txt', 'w') as file:
    json.dump(DictsList, file)
time.sleep(3)
driver.quit()
pprint(DictsList[0])

An exception occurred: Message: element click intercepted: Element <button class="button">...</button> is not clickable at point (608, 521). Other element would receive the click: <iframe id="google_ads_iframe_/71161633,22571436095/FRAGRANTICA_fragrantica/footer_sticky_0" name="google_ads_iframe_/71161633,22571436095/FRAGRANTICA_fragrantica/footer_sticky_0" title="3rd party ad content" width="1" height="1" scrolling="no" marginwidth="0" marginheight="0" frameborder="0" aria-label="Advertisement" tabindex="0" allow="attribution-reporting" data-load-complete="true" data-google-container-id="3" style="border: 0px; vertical-align: bottom; width: 728px; height: 90px;"></iframe>
  (Session info: chrome=125.0.6422.76)
Stacktrace:
	GetHandleVerifier [0x00BBB793+45827]
	(No symbol) [0x00B4DB74]
	(No symbol) [0x00A4150F]
	(No symbol) [0x00A88052]
	(No symbol) [0x00A863D8]
	(No symbol) [0x00A8425B]
	(No symbol) [0x00A83823]
	(No symbol) [0x00A785EF]
	(No symbol) [0x00AA2DFC]
	(No symbol) [0x00A78

In [9]:
DictsList = []
with open('DictsList.txt', 'r') as file:
    DictsList= json.load(file)
print(len(DictsList))

30


In [7]:
#Get in each perfume's page to get the it's details
perfumesDictsList=[]
for i in range(len(DictsList)):#update this later len(DictsList)
    try:
        driver = webdriver.Chrome(service=s, options=o)
        driver.get(DictsList[i]["url"])
        time.sleep(10)
    except:
        break
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
###################get the URL####################
    perfumeUrl=DictsList[i]["url"]

################get the Name######################
    perfumeName = soup.find_all("div", class_="cell small-12")[3].find_all("b")[0].get_text()

################get the designer##################
    perfumeDesigner = soup.find_all("div", class_="cell small-12")[3].find_all("b")[1].get_text()

######################get the img#################
    perfumeImage = soup.find_all("div", class_="cell small-12")[1].find("img")["src"]

######################get the gender##############
    perfumeGender = soup.find("small").get_text()
    try:
        perfumeRating = float(soup.find("p", class_="info-note").find_all("span")[0].get_text())
    except:
        perfumeRating = "Null"

######################get the Votes count#########
    try:
        perfumeVotesCount = int(soup.find("p", class_="info-note").find_all("span")[2].get_text().replace(',', ''))
    except:
        perfumeVotesCount = "Null"

##################find the description############
    try:
        perumeDescription = soup.find_all("div", class_="cell small-12")[3].get_text().split('Read about this perfume')[0]
    except:
        perumeDescription = "Null"

##################find the accords###############
    try:
        perfumeAccords = soup.find_all("div", class_="cell accord-box")
        perfumeAccordsDict = {}
        for i in range(len(perfumeAccords)):
            accordName = perfumeAccords[i].get_text()
            accordValue = float(perfumeAccords[i].find("div", class_="accord-bar")["style"].rsplit("width: ")[1].strip("%;"))
            perfumeAccordsDict[accordName] = accordValue
    except:
            perfumeAccordsDict = {}

#################find the notes#################
    perfumeNotes = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; text-align: center; flex-flow: wrap; align-items: flex-end; padding: 0.5rem;"})
    if len(perfumeNotes) == 3:
        i = 2
        perfumeTopNotes = []
        perfumeMidNotes = []
        perfumeBaseNotes = []
        for j in range(len(perfumeNotes[0].find_all("span", class_="link-span"))):
            perfumeTopNotes.append(perfumeNotes[0].find_all("div")[i].get_text())
            i += 3
        i = 2
        for j in range(len(perfumeNotes[1].find_all("span", class_="link-span"))):
            perfumeMidNotes.append(perfumeNotes[1].find_all("div")[i].get_text())
            i += 3
        i = 2
        for j in range(len(perfumeNotes[2].find_all("span", class_="link-span"))):
            perfumeBaseNotes.append(perfumeNotes[2].find_all("div")[i].get_text())
            i += 3
    elif len(perfumeNotes) == 2:
        i = 2
        perfumeTopNotes = []
        perfumeMidNotes = []
        perfumeBaseNotes = []
        for j in range(len(perfumeNotes[0].find_all("span", class_="link-span"))):
            perfumeTopNotes.append(perfumeNotes[0].find_all("div")[i].get_text())
            i += 3
        i = 2
        for j in range(len(perfumeNotes[1].find_all("span", class_="link-span"))):
            perfumeMidNotes.append(perfumeNotes[1].find_all("div")[i].get_text())
            i += 3              
    elif len(perfumeNotes) == 1:
        i = 2
        perfumeTopNotes = []
        perfumeMidNotes = []
        perfumeBaseNotes = []
        for j in range(len(perfumeNotes[0].find_all("span", class_="link-span"))):
            perfumeMidNotes.append(perfumeNotes[0].find_all("div")[i].get_text())
            i += 3      
    else:
        perfumeTopNotes = []
        perfumeMidNotes = []
        perfumeBaseNotes = []

#################find the Max voting#################
    voting = soup.find_all("div", class_="cell small-1 medium-1 large-1")
    # Define dictionaries to map index to label for each category
    labels = {
    "Longevity": {0: "very weak", 1: "weak", 2: "moderate", 3: "long lasting", 4: "eternal"},
    "Sillage": {0: "intimate", 1: "moderate", 2: "strong", 3: "enormous"},
    "Gender": {0: "female", 1: "more female", 2: "unisex", 3: "more male", 4: "male"},
    "PriceValue": {0: "way over", 1: "over", 2: "ok", 3: "good", 4: "great"}
    }
    maxLabels = {}
    # Iterate over each category
    for category, labelMap in labels.items():
        # Find the index of the maximum value in the voting list for the current category
        maxIndex = max(range(len(labelMap)), key=lambda i: int(voting[i].get_text()))
        # Retrieve the corresponding label from the label map
        maxLabel = labelMap[maxIndex]
        # Store the category and its corresponding label with maximum votes in the maxLabels dictionary
        maxLabels[category] = maxLabel

############prepare the json and write it to the file###############
# creating each perfume's data in a json object
    perfumeDict = {
                "url": perfumeUrl,
                "name": perfumeName,
                "desiger": perfumeDesigner,
                "image": perfumeImage,
                "gender": perfumeGender,
                "rating": perfumeRating,
                "votes count": perfumeVotesCount,
                "description": perumeDescription,
                "accords": perfumeAccordsDict,
                "top notes": perfumeTopNotes,
                "mid notes": perfumeMidNotes,
                "base notes": perfumeBaseNotes,
                "max votes": maxLabels
            }
    perfumesDictsList.append(perfumeDict)
# writing a copy of the perfume object to the json file
    with open('perfumesDictsList.txt', 'w') as file:
        json.dump(perfumesDictsList, file)
    driver.quit()
    time.sleep(10)

In [11]:
pprint(perfumesDictsList[0])

{'accords': {'amber': 76.5896,
             'cinnamon': 64.4071,
             'fresh spicy': 47.5396,
             'fruity': 44.5027,
             'sweet': 100.0,
             'vanilla': 77.966,
             'warm spicy': 79.0352,
             'woody': 48.1092},
 'base notes': ['Vanilla',
                'Tonka Bean',
                'Amberwood',
                'Myrrh',
                'Benzoin',
                'Akigalawood'],
 'description': 'Khamrah by Lattafa Perfumes is a Aromatic Spicy fragrance for '
                'women and men. This is a new fragrance. Khamrah was launched '
                'in 2022. Top notes are Cinnamon, Nutmeg and Bergamot; middle '
                'notes are Dates, Praline, Tuberose and Mahonial; base notes '
                'are Vanilla, Tonka Bean, Amberwood, Myrrh, Benzoin and '
                'Akigalawood. Lattafa Khamrah is a luxurious oriental-spicy '
                'unisex perfume that combines precious spices, the warmth of '
                

In [ ]:
########################## pre processing###################

In [16]:
import pandas as pd
# Load the data from file
with open('perfumesData.txt', 'r') as file:
    data = json.load(file)

# Create a DataFrame
df = pd.DataFrame(data)

# Standardize column names
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

# Rename any specific columns to maintain consistency
df.rename(columns={'votes_count': 'votes_count', 'desiger': 'designer'}, inplace=True)

# Handle missing values
df.fillna({'rating': 0, 'votes_count': 0}, inplace=True)

# Convert ratings and votes_count to appropriate types
df['rating'] = df['rating'].astype(float)
df['votes_count'] = df['votes_count'].astype(int)

# Extract top, mid, and base notes into individual columns
df['top_notes'] = df['top_notes'].apply(lambda x: ', '.join(x))
df['mid_notes'] = df['mid_notes'].apply(lambda x: ', '.join(x))
df['base_notes'] = df['base_notes'].apply(lambda x: ', '.join(x))

# Extract accords into individual columns
accords_df = pd.json_normalize(df['accords'])
accords_df.columns = [f'accord_{col}' for col in accords_df.columns]
# Convert accords values to float and round to 2 decimal points
accords_df = accords_df.astype(float).round(2)
# Merge accords back into the main DataFrame
df = pd.concat([df.drop(columns=['accords']), accords_df], axis=1)

# Flatten the nested dictionary into separate columns
max_votes_df = pd.json_normalize(df['max_votes'])
# Concatenate the flattened DataFrame with the original DataFrame
df = pd.concat([df.drop(columns=['max_votes']), max_votes_df], axis=1)

# Handle missing values
df.fillna(0, inplace=True)

# Drop unneaded columns, rows
df.drop(columns=['url','image', 'description'],inplace=True)
df = df.drop_duplicates(subset=['name'])

# Save preprocessed data to a new file
df.to_csv('preprocessed_perfumesData.csv', index=False)

# Print summary to check the preprocessing
df.head()

,name,designer,gender,rating,votes_count,top_notes,mid_notes,base_notes,accord_sweet,accord_warm spicy,...,accord_terpenic,accord_alcohol,accord_mineral,accord_savory,accord_whiskey,accord_conifer,Longevity,Sillage,Gender,PriceValue
0,Khamrah,Lattafa Perfumes,for women and men,4.39,8032,"Cinnamon, Nutmeg, Bergamot","Dates, Praline, Tuberose, Mahonial","Vanilla, Tonka Bean, Amberwood, Myrrh, Benzoin...",100.00,78.95,...,0.0,0.0,0.0,0.0,0.0,0.0,long lasting,enormous,more male,good
1,Club de Nuit Intense Man,Armaf,for men,4.17,15934,"Lemon, Pineapple, Bergamot, Black Currant, Apple","Birch, Jasmine, Rose","Musk, Ambergris, Patchouli, Vanilla",52.47,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,long lasting,enormous,more male,good
2,9pm,Afnan,for men,4.51,6021,"Apple, Cinnamon, Wild Lavender, Bergamot","Orange Blossom, Lily-of-the-Valley","Vanilla, Tonka Bean, Amber, Patchouli",54.72,58.99,...,0.0,0.0,0.0,0.0,0.0,0.0,long lasting,enormous,more male,good
3,Hawas for Him,Rasasi,for men,4.39,5065,"Apple, Bergamot, Lemon, Cinnamon","Watery Notes, Plum, Cardamon, Orange Blossom","Ambergris, Musk, Patchouli, Driftwood",58.33,71.22,...,0.0,0.0,0.0,0.0,0.0,0.0,long lasting,enormous,more male,good
4,Asad,Lattafa Perfumes,for men,4.29,4703,"Black Pepper, Tobacco, Pineapple","Patchouli, Coffee, Iris","Vanilla, Amber, Dry Wood, Benzoin, Labdanum",69.49,75.65,...,0.0,0.0,0.0,0.0,0.0,0.0,long lasting,enormous,more male,good


In [18]:
import pandas as pd

# Replace 'perfumesData.csv' with your CSV file path
csv_file = 'preprocessed_perfumesData.csv'
excel_file = 'preprocessed_perfumesData.xlsx'

# Read the CSV file
df = pd.read_csv(csv_file)

# Write to an Excel file
df.to_excel(excel_file, index=False)

print(f"CSV file '{csv_file}' has been successfully converted to Excel file '{excel_file}'")


CSV file 'preprocessed_perfumesData.csv' has been successfully converted to Excel file 'preprocessed_perfumesData.xlsx'
